In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import warnings
warnings.simplefilter(action='ignore')

In [52]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
data = pd.read_csv(url, names=columns)

X = data.drop('species', axis=1).values
y = data['species'].values

y = LabelEncoder().fit_transform(y)
y = torch.tensor(y, dtype=torch.long)
X = torch.tensor(X, dtype=torch.float32)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

class neuroverkko(nn.Module):
    def __init__(self):
        super(neuroverkko, self).__init__()
        self.hidden = nn.Linear(4, 8)
        self.output = nn.Linear(8, 3)
        
    def forward(self, x):
        x = F.relu(self.hidden(x))
        x = F.softmax(self.output(x), dim=1)
        return X
    
model = neuroverkko()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [50]:
for epoch in range(50):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, torch.max(batch_y, 1)[1])
        loss.backward()
        optimizer.step()
        
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            _, predicted = torch.max(outputs, 1)
            _, labels = torch.max(batch_y, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Accuracy: {100*correct/total}%')

ValueError: Expected input batch_size (150) to match target batch_size (16).